In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt


In [ ]:
model_url = "https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2"

IMAGE_SHAPE = (224, 224)

layer = hub.KerasLayer(model_url)
model = tf.keras.Sequential([layer])

In [ ]:
def extract2(file):
  file = Image.open(file).convert('L').resize(IMAGE_SHAPE)
  #display(file)

  file = np.stack((file,)*3, axis=-1)

  file = np.array(file)/255.0
  print(file.shape)

  embedding = model.predict(file[np.newaxis, ...])
  #print(embedding)
  vgg16_feature_np = np.array(embedding)
  flattended_feature = vgg16_feature_np.flatten()

  #print(len(flattended_feature))
  #print(flattended_feature)
  #print('-----------')
  return flattended_feature

In [ ]:
def get_image(file):
  file = Image.open(file).convert('L').resize(IMAGE_SHAPE)
  file = np.stack((file,)*3, axis=-1)
  file = np.array(file)/255.0
  return file

In [ ]:
def get_sobel_image(image):
  # Get x-gradient in "sx"
  sx = ndimage.sobel(image,axis=0,mode='constant')
  # Get y-gradient in "sy"
  sy = ndimage.sobel(image,axis=1,mode='constant')
  # Get square root of sum of squares
  return np.hypot(sx,sy)

In [ ]:
def extract_features(image):

  embedding = model.predict(image[np.newaxis, ...])
  #print(embedding)
  vgg16_feature_np = np.array(embedding)
  flattended_feature = vgg16_feature_np.flatten()

  #print(len(flattended_feature))
  #print(flattended_feature)
  #print('-----------')
  return flattended_feature

In [ ]:
im1=get_image(r"C:\Data\Ramen\FOTOS\window1-1.jpg")
im1_lines=get_sobel_image(im1)
im1_features=extract_features(im1)
im1_features_lines=extract_features(im1_lines)

im2=get_image(r"C:\Data\Ramen\FOTOS\window1-2.jpg")
im2_lines=get_sobel_image(im2)
im2_features=extract_features(im2)
im2_features_lines=extract_features(im2_lines)


In [ ]:
f = plt.figure()

#subplot(r,c) provide the no. of rows and columns
f, axarr = plt.subplots(2,2) 
# use the created array to output your multiple images. In this case I have stacked 4 images vertically
axarr[0,0].imshow(im1,cmap=plt.cm.gray)
axarr[0,1].imshow(im1_lines)
axarr[1,0].imshow(im2)
axarr[1,1].imshow(im2_lines)

In [ ]:
from scipy.spatial import distance

In [ ]:
metric = 'cosine'

In [ ]:
dc = distance.cdist([im1_features], [im2_features], metric)[0]
print(dc)
print("the distance between original and the original is {}".format(dc))

In [ ]:
dc = distance.cdist([im1_features_lines], [im2_features_lines], metric)[0]
print(dc)
print("the distance between original and the original is {}".format(dc))

In [ ]:
dc = distance.cdist([im1_features], [im1_features_lines], metric)[0]
print(dc)
print("the distance between original and the original is {}".format(dc))

In [ ]:
dc = distance.cdist([im1_features], [im1_features], metric)[0]
print(dc)
print("the distance between original and the original is {}".format(dc))

In [ ]:
from tensorflow.keras import layers
# Embedding output layer with L2 norm
from tensorflow_similarity.layers import MetricEmbedding 
# Specialized metric loss
from tensorflow_similarity.losses import MultiSimilarityLoss 
# Sub classed keras Model with support for indexing
from tensorflow_similarity.models import SimilarityModel
# Data sampler that pulls datasets directly from tf dataset catalog
from tensorflow_similarity.samplers import TFDatasetMultiShotMemorySampler
# Nearest neighbor visualizer
from tensorflow_similarity.visualization import viz_neigbors_imgs
